## Liquidity Stress Engine

This is a lightweight, simplified interactive liquidity stress testing tool used to assess whether a fund can meet redemption requirements under a range of market stress scenarios, including equity drawdowns, credit spread shocks, FX movements, and elevated redemption activity. Liquidity is modeled at the bucket level with assumed liquidation timelines and stress impacts applied to market values and cash demands. 

Note: Model assumptions are illustrative and intentionally simplified. They are not calibrated to any specific fund, strategy, or market environment, and should not be interpreted as precise or predictive estimates.

In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))

In [2]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

from engine.assets import default_liquidity_profile
from engine.liquidity import run_waterfall, apply_stress
from engine.metrics import liquidity_metrics
from engine.assumptions import ASSUMPTIONS
from engine.scenario_translation import translate_scenario


In [3]:
# Cosmetics and HTML Formatting

DARK_BG = "#0f1115"     # charcoal black
CARD_BG = "#151922"     # dark slate
TEXT_MAIN = "#e6e6e6"   # light grey
TEXT_MUTED = "#9aa0a6"  # slate grey
GREEN = "#2ecc71"       # green
RED = "#e74c3c"         # red
AMBER = "#f1c40f"       # golden yellow
BORDER = "#2a2f38"      # dark blue/grey

HTML("""
<style>
/* page background */
body {
    background-color: #0f1115 !important;
    color: #e6e6e6;
}

/* voila and notebook containers */
.voila-app,
.jp-Notebook,
.jp-Cell,
.jp-Cell-outputWrapper {
    background-color: #0f1115 !important;
}

/* widget areas */
.widget-area,
.jupyter-widgets,
.jupyter-widget-box {
    background-color: #0f1115 !important;
}

/* widget text */
.widget-label,
.widget-readout,
.widget-slider-label,
.widget-checkbox label,
.widget-radio label {
    color: #e6e6e6 !important;
}

/* slider min/max values */
.noUi-value,
.noUi-tooltip {
    color: #e6e6e6 !important;
}

/* tables */
table {
    background-color: #151922;
    color: #e6e6e6;
}

/* remove any white */
.output,
.output_area {
    background-color: #0f1115 !important;
}
</style>
""")


In [4]:
# Helpers and Plotting Functions

def fmt_bn(x):
    return f"${x / 1e9:,.2f} bn"


def color_cash_used(val): 
    if "$0.00" in val: 
        return "color:#999"
    return "color:#1a7f37; font-weight:600" 


def color_days(val): 
    if val <= 1: 
        return "color:#1a7f37; font-weight:600" 
    elif val <= 30: 
        return "color:#b26a00"
    else: return "color:#b42318; font-weight:600"


def render_summary(metrics, profile, scenario):
    status = "BREACH" if metrics["breach"] else "PASS"
    status_color = RED if metrics["breach"] else GREEN

    return widgets.HTML(f"""
    <div style="
        display:flex;
        gap:16px;
        margin-bottom:16px;
        color:{TEXT_MAIN};
    ">
        <div style="flex:1; border:1px solid {BORDER}; padding:12px; background:{CARD_BG};">
            <div style="color:{TEXT_MUTED}; font-size:12px;">Liquidity Status</div>
            <div style="font-size:20px; font-weight:600; color:{status_color};">{status}</div>
        </div>

        <div style="flex:1; border:1px solid {BORDER}; padding:12px; background:{CARD_BG};">
            <div style="color:{TEXT_MUTED}; font-size:12px;">Liquidity Coverage</div>
            <div style="font-size:20px; font-weight:600;">{metrics['liquidity_coverage']:.2f}x</div>
        </div>

        <div style="flex:1; border:1px solid {BORDER}; padding:12px; background:{CARD_BG};">
            <div style="color:{TEXT_MUTED}; font-size:12px;">Days to Liquidity</div>
            <div style="font-size:20px; font-weight:600;">{metrics['days_to_liquidity']}</div>
        </div>

        <div style="flex:1; border:1px solid {BORDER}; padding:12px; background:{CARD_BG};">
            <div style="color:{TEXT_MUTED}; font-size:12px;">Total Fund Value</div>
            <div style="font-size:20px; font-weight:600;">{fmt_bn(profile['market_value'].sum())}</div>
        </div>
    </div>

    <div style="color:{TEXT_MUTED}; font-size:13px; margin-bottom:12px;">
        Scenario — Equity {scenario['equity_drawdown']:.0%},
        Credit {scenario['credit_shock']:.0%},
        FX {scenario['fx_shock']:.0%},
        Redemption {scenario['redemption']:.0%}
    </div>
    """)


def plot_liquidity_charts(metrics, waterfall):
    fig, axes = plt.subplots(1, 2, figsize=(10, 3.5))
    fig.patch.set_facecolor(DARK_BG)

    for ax in axes:
        ax.set_facecolor(CARD_BG)
        ax.tick_params(colors=TEXT_MUTED, labelsize=9)
        ax.xaxis.label.set_color(TEXT_MUTED)
        ax.yaxis.label.set_color(TEXT_MUTED)
        ax.title.set_color(TEXT_MAIN)

        for spine in ax.spines.values():
            spine.set_color(BORDER)

    axes[0].bar(
        ["Cash Required", "Cash Raised"],
        [metrics["cash_required"] / 1e9, metrics["cash_raised"] / 1e9],
        color=[RED, GREEN]
    )
    axes[0].set_title("Liquidity Coverage", color=TEXT_MAIN)
    axes[0].set_ylabel("CAD (bn)", color=TEXT_MUTED)

    wf = waterfall.sort_values("days_to_cash")
    axes[1].barh(
        wf["bucket"],
        wf["cash_used"] / 1e9,
        color=GREEN
    )
    axes[1].set_title("Liquidity Waterfall", color=TEXT_MAIN)
    axes[1].set_xlabel("Cash Raised (CAD bn)", color=TEXT_MUTED)

    plt.tight_layout()
    plt.show()

In [5]:
# Sliders and Checkbox Controls

equity_slider = widgets.FloatSlider(
    value = 0.0,
    min = 0.0,
    max = 0.99,
    step = 0.01,
    description = "Equity Drawdown",
    readout_show = True,
    readout_format = "0.0%",
    style = {"description_width": "150px"},
    layout = widgets.Layout(width = "280px")
)

credit_slider = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=0.99,
    step=0.01,
    description="Credit Shock",
    readout_format="0.0%",
    style = {"description_width": "150px"},
    layout = widgets.Layout(width = "280px")
)

fx_slider = widgets.FloatSlider(
    value=0.0,
    min=0.0,
    max=0.99,
    step=0.01,
    description="FX Shock",
    readout_format="0.0%",
    style = {"description_width": "150px"},
    layout = widgets.Layout(width = "280px")
)

redemption_slider = widgets.FloatSlider(
    value = 0.0,
    min = 0.0,
    max = 0.99,
    step = 0.01,
    description = "Fund Redemption",
    readout_show = True,
    readout_format = "0.0%",
    style = {"description_width": "150px"},
    layout = widgets.Layout(width = "280px")
)

freeze_cash = widgets.Checkbox(value=False, description="Freeze Cash & ST Bonds")
freeze_t1 = widgets.Checkbox(value=False, description="Freeze Public Equities [T+1]")
freeze_t5 = widgets.Checkbox(value=False, description="Freeze Public Credit [T+5]")
freeze_t30 = widgets.Checkbox(value=False, description="Freeze Real Estate [T+30]")
freeze_t90 = widgets.Checkbox(value=False, description="Freeze Infrastructure [T+90]")

controls = widgets.VBox(
    [widgets.HTML(f"<h3 style='color:{TEXT_MAIN}; margin-bottom:8px;'>Stress Inputs</h3>"), equity_slider, credit_slider, fx_slider, redemption_slider, freeze_cash, freeze_t1, freeze_t5, freeze_t30, freeze_t90,],
    layout = widgets.Layout(width="360px", padding="8px",),
)

output = widgets.Output()

In [6]:
# Core function to run the test

def run_stress(_ = None):
    with output:
        clear_output()
        profile = default_liquidity_profile()
        if freeze_cash.value:
            profile.loc[profile["bucket"] == "Cash & Short-Term Bonds", "available"] = False
        if freeze_t1.value:
            profile.loc[profile["bucket"] == "Public Equities [T+1]", "available"] = False
        if freeze_t5.value:
            profile.loc[profile["bucket"] == "Public Credit [T+5]", "available"] = False
        if freeze_t30.value:
            profile.loc[profile["bucket"] == "Real Estate [T+30]", "available"] = False
        if freeze_t90.value:
            profile.loc[profile["bucket"] == "Infrastructure [T+90]", "available"] = False

        scenario = {
            "equity_drawdown": -equity_slider.value,
            "credit_shock": -credit_slider.value,
            "fx_shock": -fx_slider.value,
            "redemption": redemption_slider.value,
        }

        stressed_profile, liquidity_demand = translate_scenario(profile = profile, scenario = scenario, assumptions = ASSUMPTIONS,)
        stressed_profile = apply_stress(stressed_profile)
        waterfall, summary = run_waterfall(stressed_profile=stressed_profile, cash_required=liquidity_demand,)
        metrics = liquidity_metrics(summary)
        display(render_summary(metrics, profile, scenario))

        waterfall_display = waterfall.copy()
        for col in ["stressed_value", "cash_used", "remaining_value"]:
            waterfall_display[col] = waterfall_display[col].apply(fmt_bn)

        waterfall_display = waterfall_display.reset_index(drop=True)
        waterfall_display = waterfall_display.rename(columns={
            "bucket": "Asset Bucket",
            "stressed_value": "Stressed Value",
            "cash_used": "Cash Used",
            "remaining_value": "Remaining Value",
            "days_to_cash": "Days to Cash"
        })

        styled = waterfall_display.style \
            .set_properties(**{"text-align": "right"}) \
            .set_table_styles([
                {"selector": "th", "props": [("background", "#f2f2f2")]},
                {"selector": "th", "props": [("text-align", "right")]},
            ])

        widgets.HTML(f"""
        <h3 style="color:{TEXT_MAIN}; margin-bottom:8px;">
        Liquidity Waterfall (Post-Stress)
        </h3>
        <p style="color:{TEXT_MUTED}; margin-top:0;">
        Order reflects liquidity speed from fastest to slowest.
        </p>
        """)

        styled = (
            waterfall_display.style
                .set_properties(**{
                    "text-align": "right",
                    "color": TEXT_MAIN,
                    "background-color": CARD_BG,
                })
                .set_properties(
                    subset=["Asset Bucket"],
                    **{"text-align": "left", "color": TEXT_MAIN}
                )
                .map(color_cash_used, subset=["Cash Used"])
                .map(color_days, subset=["Days to Cash"])
                .set_table_styles([
                    {"selector": "th", "props": [
                        ("background", "#1d2230"),
                        ("color", TEXT_MAIN),
                        ("font-weight", "600"),
                    ]},
                    {"selector": "td", "props": [
                        ("color", TEXT_MAIN),
                        ("border-bottom", f"1px solid {BORDER}"),
                    ]},
                ])
        )

        display(styled)
        plot_liquidity_charts(metrics, waterfall)

In [7]:
# Looping, checking for user inputs

for w in [equity_slider, credit_slider, fx_slider, redemption_slider, freeze_cash, freeze_t1, freeze_t5, freeze_t30, freeze_t90,]: w.observe(run_stress, names = "value")
app = widgets.HBox([controls, output], layout = widgets.Layout(background_color=DARK_BG, padding="12px"))
display(app)
run_stress()